Snakes

In [3]:
import pandas as pd
import re

trash_df = pd.read_csv('../data/trash_hauler_report.csv')
#Explore the data
print(trash_df.shape)
print(trash_df.columns)
print(trash_df.dtypes)
print(trash_df.info())
trash_df.head()


(20226, 11)
Index(['Request Number', 'Date Opened', 'Request ', 'Description',
       'Incident Address', 'Zip Code', 'Trash Hauler', 'Trash Route',
       'Council District', 'State Plan X', 'State Plan Y'],
      dtype='object')
Request Number        int64
Date Opened          object
Request              object
Description          object
Incident Address     object
Zip Code            float64
Trash Hauler         object
Trash Route          object
Council District    float64
State Plan X        float64
State Plan Y        float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip 

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,11/01/17,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


In [5]:
#I noticed extra spaces in some of the raw data, so this will clean up leading/trailing and extra spaces.

#Clean only columns with object (string) type
str_cols = trash_df.select_dtypes(include='object').columns

#Clean and normalize whitespace in each string column
for col in str_cols:
    trash_df[col] = trash_df[col].astype(str).apply(lambda x: ' '.join(x.split()))

#Clean column whitespace
trash_df.columns = trash_df.columns.str.strip()



In [7]:
#Make 'Request' column into a category
trash_df['Request'] = trash_df['Request'].astype('category')
trash_df['Request'].cat.categories
trash_df['Request'].value_counts()


Request
Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2629
Trash Collection Complaint               2312
Damage to Property                        257
Name: count, dtype: int64

In [9]:
#make all lowercase so matches are consistent
trash_df['description_clean'] = trash_df['Description'].str.lower()

In [11]:
#To help ensure data is complete, this creates a new Street Name column with only the street names (no address numbers)
#Then if data is missing (Zip, Hauler, Route, District etc) it will complete the missing data based on the assumption that all addresses on the same street
#uses the same Zip, Hauler, Route, District etc). It will complete any missing data for Incidents on the same Street Name.
#It normalizes the new Street Name to USPS naming conventions so they can be easily grouped if needed.

# USPS street suffix abbreviations
suffix_map = {
    'avenue': 'Ave', 'av': 'Ave', 'av.': 'Ave',
    'road': 'Rd', 'rd': 'Rd',
    'street': 'St', 'st': 'St',
    'boulevard': 'Blvd', 'blvd': 'Blvd',
    'drive': 'Dr', 'dr': 'Dr',
    'court': 'Ct', 'ct': 'Ct',
    'lane': 'Ln', 'ln': 'Ln',
    'place': 'Pl', 'pl': 'Pl',
    'circle': 'Cir', 'cir': 'Cir',
    'trail': 'Trl', 'trl': 'Trl',
    'parkway': 'Pkwy', 'pkwy': 'Pkwy',
    'terrace': 'Ter', 'ter': 'Ter',
    'way': 'Way',
    'loop': 'Loop'
}

# USPS directional abbreviations
direction_map = {
    'north': 'N', 'n.': 'N', 'n': 'N',
    'south': 'S', 's.': 'S', 's': 'S',
    'east': 'E', 'e.': 'E', 'e': 'E',
    'west': 'W', 'w.': 'W', 'w': 'W'
}

#Full address normalization (keeps number)
def normalize_address(address):
    if pd.isnull(address):
        return None
    address = str(address).strip().lower()
    words = address.split()
    result = []
    for word in words:
        clean_word = re.sub(r'[^\w]', '', word)
        if clean_word in direction_map:
            result.append(direction_map[clean_word])
        elif clean_word in suffix_map:
            result.append(suffix_map[clean_word])
        else:
            result.append(clean_word.capitalize())
    return ' '.join(result)

#Street name normalization (removes house number first)
def normalize_street(address):
    if pd.isnull(address):
        return None
    address = str(address).strip()
    # Remove leading house number
    address_no_number = re.sub(r'^\s*\d+[^\s]*\s+', '', address)
    return normalize_address(address_no_number)

#Apply to DataFrame
trash_df['Street Name'] = trash_df['Incident Address'].apply(normalize_street)
trash_df['Incident Address'] = trash_df['Incident Address'].apply(normalize_address)

In [13]:
#I noticed that several other descriptions mention missed pickups in the Description column.
#This is a list of keywords it finds in the event that the Request is not properly labeled as a Missed Pickup.
#More keywords could be added if noticed in the raw data.

import re

keywords = [
    'miss', 'missed', 'not picked up', 'not collected', 'no pickup',
    'never came', 'did not collect', 'didn’t pick up', 'did not pick up'
]

pattern = '|'.join(re.escape(word) for word in keywords)

is_missed_request = trash_df['Request'].str.contains("missed pickup", case=False, na=False)
is_missed_description = trash_df['description_clean'].str.contains(pattern, na=False)

missed_pickups = trash_df[is_missed_request | is_missed_description]


In [15]:
#I noticed that some entries seemed to be duplicates based on the date opened and the Incident Address in the raw data.
#This filters out any Requests that may have been submitted multiple times for the same Date/Incident Address
missed_pickups = missed_pickups.copy()
missed_pickups['Date Opened'] = pd.to_datetime(
    missed_pickups['Date Opened'], format='%m/%d/%y', errors='coerce'
)

unique_missed = missed_pickups.drop_duplicates(subset=['Incident Address', 'Date Opened'])

In [17]:
#I noticed some data was missing, i.e. Zip Code, Trash Hauler, Route, District etc.
#This groups by the newly created 'Street Name' column and uses the infer data for the Street Name to fill in incomplete data if other instances of the same Street.
cols_to_fill = ['Zip Code', 'Trash Hauler', 'Trash Route', 'Council District']

for col in cols_to_fill:
    trash_df[col] = trash_df.groupby('Street Name')[col].transform(lambda x: x.ffill().bfill())


In [18]:
#This exports the cleaned data to a CSV file. The output only includes unique missed pickups based on Request and/or keywords in Description columns.

#Filter based on Request type or keywords in Description
keywords = [
    'miss', 'missed', 'not picked up', 'not collected', 'no pickup',
    'never came', 'did not collect', 'didn’t pick up', 'did not pick up'
]
pattern = '|'.join(re.escape(word) for word in keywords)

is_missed_request = trash_df['Request'].str.contains("missed pickup", case=False, na=False)
is_missed_description = trash_df['description_clean'].str.contains(pattern, na=False)

missed_pickups = trash_df[is_missed_request | is_missed_description].copy()

#Normalize data
missed_pickups['Date Opened'] = pd.to_datetime(missed_pickups['Date Opened'], format='%m/%d/%y', errors='coerce')

#Remove same-day duplicates
unique_missed = missed_pickups.drop_duplicates(subset=['Incident Address', 'Date Opened']).copy()

#Normalize Street Name
unique_missed['Street Name'] = unique_missed['Incident Address'].apply(normalize_street)

#Fill missing data based on same Street Names
cols_to_fill = ['Zip Code', 'Trash Hauler', 'Trash Route', 'Council District']
for col in cols_to_fill:
    unique_missed[col] = unique_missed.groupby('Street Name')[col].transform(lambda x: x.ffill().bfill())
    
#Export to /data folder
unique_missed.to_csv('../data/cleaned_missed_pickups.csv', index=False)

In [20]:
print(trash_df.columns)

Index(['Request Number', 'Date Opened', 'Request', 'Description',
       'Incident Address', 'Zip Code', 'Trash Hauler', 'Trash Route',
       'Council District', 'State Plan X', 'State Plan Y', 'description_clean',
       'Street Name'],
      dtype='object')
